In [4]:
import json,codecs
import urllib.request
import urllib.parse

In [5]:
def getDataWaterLevelNow(province_name="นครศรีธรรมราช", amphoe_name="ทุ่งสง"):
    url = "https://api-v3.thaiwater.net/api/v1/thaiwater30/public/waterlevel_load"
    with urllib.request.urlopen(url) as response:
        data = json.loads(response.read())
    data = data["waterlevel_data"]["data"]
    # data
    len(data)
    def thungsongFilter(item):
        # print(item)
        return (item["geocode"]["amphoe_name"]["th"] == "ทุ่งสง") and (item["geocode"]["province_name"]["th"] == "นครศรีธรรมราช" )
        # return (item["geocode"]["province_name"]["th"] == "นครศรีธรรมราช" )

    thungsongData = filter(thungsongFilter,data)
    thungsongData = list(thungsongData)
    return thungsongData

wl = getDataWaterLevelNow(province_name="นครศรีธรรมราช", amphoe_name="ทุ่งสง")
with codecs.open('data/now-wl.json', 'w', encoding='utf-8') as f:
    json.dump(wl, f, ensure_ascii=False)
    print("Write file data/now-wl.json")

Write file data/now-wl.json


In [6]:
def getDataRainNow(province_name="นครศรีธรรมราช", amphoe_name="ทุ่งสง"):
    url = "https://api-v3.thaiwater.net/api/v1/thaiwater30/public/rain_24h"
    with urllib.request.urlopen(url) as response:
        data = json.loads(response.read())
    data = data["data"]
    # data
    len(data)
    def thungsongFilter(item):
        # print(item)
        return (item["geocode"]["amphoe_name"]["th"] == "ทุ่งสง") and (item["geocode"]["province_name"]["th"] == "นครศรีธรรมราช" )
        # return (item["geocode"]["province_name"]["th"] == "นครศรีธรรมราช" )

    thungsongData = filter(thungsongFilter,data)
    thungsongData = list(thungsongData)
    return thungsongData

rain = getDataRainNow(province_name="นครศรีธรรมราช", amphoe_name="ทุ่งสง")
with codecs.open('data/now-rain.json', 'w', encoding='utf-8') as f:
    json.dump(rain, f, ensure_ascii=False)    
    print("Write file data/now-rain.json")

Write file data/now-rain.json


In [44]:
import datetime
import re
def getDataWaterLevelTimeframe(station_id="795", timeframe="H1"):
    modes = {
        "H1" : 1,   #2 days => 24 hours
        "H4" : 5,   #5/7 weeks => 30 4-hours 
        "D1" : 30,  #1 months => 30 days
        "MN" : 366, #1 years => 12 months*2 
    }    
    params = {
        'station_type': 'tele_waterlevel',
        'station_id': str(station_id),
        'start_date': str(datetime.datetime.today() - datetime.timedelta(days=modes[timeframe])),
        'end_date': str(datetime.datetime.today()),
        # 'start_date': '2022-12-11',
        # 'end_date': '2022-12-14 20:12',
    }
    paramsString = urllib.parse.urlencode(params)
    url = "https://api-v3.thaiwater.net/api/v1/thaiwater30/public/waterlevel_graph"
    url = "{}?{}".format(url,paramsString)
    # url
    # https://api-v3.thaiwater.net/api/v1/thaiwater30/public/waterlevel_graph?station_type=tele_waterlevel&station_id=795&start_date=2022-12-11&end_date=2022-12-14%2020:12
    # url = "https://api-v3.thaiwater.net/api/v1/thaiwater30/public/waterlevel_load"
    with urllib.request.urlopen(url) as response:
        data = json.loads(response.read())
    data = data["data"]
    # data
    # data
    len(data)
    def makeTimeframeFilter(timeframe="H1"):
        def myTimeframeFilter(item):
            # print(item)
            if(timeframe=="H1"):
                return ":00" in item["datetime"] and item["value"] is not None    
            elif(timeframe=="H4") :
                return re.search("(0[048]|1[26]|20):00", item["datetime"]) is not None and item["value"] is not None 
            elif(timeframe=="D1") :
                return "00:00" in item["datetime"] and item["value"] is not None 
            elif(timeframe=="MN") :
                return re.search("(01|16) 00:00", item["datetime"]) is not None and item["value"] is not None 
            else:
                print("else")
                return item
        return myTimeframeFilter
    timeframeFilter = makeTimeframeFilter(timeframe=timeframe)
    graph_data = filter(timeframeFilter,data["graph_data"])
    graph_data = list(graph_data)
    # get only active 24 samples
    graph_data = graph_data[-24:]
    data["graph_data"] = graph_data
    return data

#H1
wl = getDataWaterLevelTimeframe(station_id="795", timeframe="H1")
with codecs.open('data/wl-timeframe-H1.json', 'w', encoding='utf-8') as f:
    json.dump(wl, f, ensure_ascii=False)
    print("Write file data/data/wl-timeframe-H1.json")
#H4
wl = getDataWaterLevelTimeframe(station_id="795", timeframe="H4")
with codecs.open('data/wl-timeframe-H4.json', 'w', encoding='utf-8') as f:
    json.dump(wl, f, ensure_ascii=False)
    print("Write file data/data/wl-timeframe-H4.json")
#D1
wl = getDataWaterLevelTimeframe(station_id="795", timeframe="D1")
with codecs.open('data/wl-timeframe-D1.json', 'w', encoding='utf-8') as f:
    json.dump(wl, f, ensure_ascii=False)
    print("Write file data/data/wl-timeframe-D1.json")
#MN
wl = getDataWaterLevelTimeframe(station_id="795", timeframe="MN")
with codecs.open('data/wl-timeframe-MN.json', 'w', encoding='utf-8') as f:
    json.dump(wl, f, ensure_ascii=False)
    print("Write file data/data/wl-timeframe-MN.json")

Write file data/data/wl-timeframe-H1.json
Write file data/data/wl-timeframe-H4.json
Write file data/data/wl-timeframe-D1.json
Write file data/data/wl-timeframe-MN.json


{'graph_data': [{'datetime': '2022-12-11 00:00',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 00:10',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 00:20',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 00:30',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 00:40',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 00:50',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 01:00',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 01:10',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 01:20',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 01:30',
   'value': 51.29,
   'value_out': None,
   'discharge': Non

In [17]:

modes = {
    "H1" : 1,   #1 days => 24 hours
    "H4" : 5,   #5/7 weeks => 30 4-hours 
    "D1" : 30,  #1 months => 30 days
    "W1" : 270, #1 years => 30 weeks 
}



{'graph_data': [{'datetime': '2022-12-11 00:00',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 00:10',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 00:20',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 00:30',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 00:40',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 00:50',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 01:00',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 01:10',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 01:20',
   'value': 51.29,
   'value_out': None,
   'discharge': None},
  {'datetime': '2022-12-11 01:30',
   'value': 51.29,
   'value_out': None,
   'discharge': Non

In [38]:
re.search("[(00)(04)(08)(12)(16)(20)]:00", "2022-12-11 20:00")

<re.Match object; span=(12, 16), match='0:00'>